In [20]:
import os
import pandas as pd
from tensorflow.python.summary.summary_iterator import summary_iterator
from tqdm import tqdm

In [31]:
test_sm = summary_iterator("/mnt/e/CODE/Github/improving_robotics_datasets/src/runs/asl_asl_prune_params_subset+02032023_20:06:19_ps-[5, 'p', 5]_gradient-gradcam_px-max-subs-0.8_pt-0.9124590689765743_cs-True/events.out.tfevents.1677784113.Eragon.27467.61")

In [44]:
for e in test_sm:
    print(e.summary.value[0])

In [52]:
def convert_tfevent(filepath):
    return pd.DataFrame([
        parse_tfevent(e, filepath) for e in summary_iterator(filepath) if len(e.summary.value)
    ])

def parse_tfevent(tfevent, filepath):
    return dict(
        fname = filepath,
        wall_time=tfevent.wall_time,
        name=tfevent.summary.value[0].tag,
        step=tfevent.step,
        value=tfevent.summary.value[0].tag,
    )

columns_order = ['fname','wall_time', 'name', 'step', 'value']

out = []
for (root, _, filenames) in os.walk("./runs/"):
    for filename in filenames:
        if "events.out.tfevents" not in filename and "prune" not in filename:
            continue
        try:
            file_full_path = os.path.join(root, filename)
            out.append(convert_tfevent(file_full_path))
        except:
            pass

# Concatenate (and sort) all partial individual dataframes
all_df = pd.concat(out)[columns_order]
all_df.reset_index(drop=True)


,fname,wall_time,name,step,value
0,./runs/asl_asl_prune_params_subset+02032023_18...,1.677778e+09,experiment_name/text_summary,0,experiment_name/text_summary
1,./runs/asl_asl_prune_params_subset+02032023_18...,1.677778e+09,ds_path/text_summary,0,ds_path/text_summary
2,./runs/asl_asl_prune_params_subset+02032023_18...,1.677778e+09,ds_name/text_summary,0,ds_name/text_summary
3,./runs/asl_asl_prune_params_subset+02032023_18...,1.677778e+09,name_fn/text_summary,0,name_fn/text_summary
4,./runs/asl_asl_prune_params_subset+02032023_18...,1.677778e+09,image_size/text_summary,0,image_size/text_summary
...,...,...,...,...,...
20048,./runs/testing_phase_resuls/asl_test_asl_start...,1.677523e+09,Loss/Train,7,Loss/Train
20049,./runs/testing_phase_resuls/asl_test_asl_start...,1.677523e+09,Acc/Train,7,Acc/Train
20050,./runs/testing_phase_resuls/asl_test_asl_start...,1.677523e+09,proxy_step,7,proxy_step
20051,./runs/testing_phase_resuls/asl_test_asl_start...,1.677523e+09,Loss/Val,7,Loss/Val


In [55]:
all_df["name"].unique()

array(['experiment_name/text_summary', 'ds_path/text_summary',
       'ds_name/text_summary', 'name_fn/text_summary',
       'image_size/text_summary', 'batch_size/text_summary',
       'epoch_steps/text_summary', 'enable_proxy_attention/text_summary',
       'change_subset_attention/text_summary', 'num_gpu/text_summary',
       'num_cpu/text_summary', 'transfer_imagenet/text_summary',
       'subset_images/text_summary', 'proxy_threshold/text_summary',
       'pixel_replacement_method/text_summary', 'model/text_summary',
       'proxy_steps/text_summary', 'load_proxy_data/text_summary',
       'gradient_method/text_summary', 'clear_every_step/text_summary',
       'fname_start/text_summary', 'device/text_summary',
       'global_run_count/text_summary', 'label_map/text_summary',
       'rev_label_map/text_summary', 'num_classes/text_summary',
       'proxy_step', 'Loss/Train', 'Acc/Train', 'Loss/Val', 'Acc/Val',
       'Number_Chosen', 'original_images', 'converted_proxy'],
      dtyp

In [62]:
all_df.sort_values(['name']).groupby('name').head(10)

,fname,wall_time,name,step,value
64,./runs/testing_phase_resuls/asl_asl_test_notra...,1.677623e+09,Acc/Train,7,Acc/Train
28,./runs/testing_phase_resuls/asl_test_asl_start...,1.677506e+09,Acc/Train,1,Acc/Train
28,./runs/testing_phase_resuls/asl_test_asl_start...,1.677521e+09,Acc/Train,1,Acc/Train
46,./runs/testing_phase_resuls/asl_test_asl_start...,1.677506e+09,Acc/Train,4,Acc/Train
31,./runs/testing_phase_resuls/asl_asl_test_notra...,1.677614e+09,Acc/Train,11,Acc/Train
...,...,...,...,...,...
11,./runs/testing_phase_resuls/asl_asl_test_notra...,1.677624e+09,transfer_imagenet/text_summary,0,transfer_imagenet/text_summary
11,./runs/testing_phase_resuls/asl_test_asl_start...,1.677508e+09,transfer_imagenet/text_summary,0,transfer_imagenet/text_summary
11,./runs/testing_phase_resuls/asl_test_asl_start...,1.677509e+09,transfer_imagenet/text_summary,0,transfer_imagenet/text_summary
11,./runs/testing_phase_resuls/asl_test_asl_start...,1.677509e+09,transfer_imagenet/text_summary,0,transfer_imagenet/text_summary


In [9]:
all_df.to_csv("collected_logs.csv")